In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn.metrics import classification_report

# --- Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
EPOCHS = 6
IMG_SIZE = 128
NUM_CLASSES = 2
EPSILON = 0.03
PGD_STEPS = 7

In [10]:
# --- Adversarial Attack Functions ---
def fgsm_attack(model, images, labels, epsilon=EPSILON):
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbed = images + epsilon * images.grad.sign()
    return torch.clamp(perturbed, 0, 1).detach()

def pgd_attack(model, images, labels, epsilon=EPSILON, alpha=0.01, iters=PGD_STEPS):
    perturbed = images.clone().detach()
    for _ in range(iters):
        perturbed.requires_grad = True
        outputs = model(perturbed)
        loss = F.cross_entropy(outputs, labels)
        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            perturbed += alpha * perturbed.grad.sign()
            perturbed = torch.max(torch.min(perturbed, images + epsilon), images - epsilon)
            perturbed = torch.clamp(perturbed, 0, 1)
    return perturbed.detach()

def one_pixel_attack(images, pixel_count=1):
    perturbed = images.clone()
    batch_size, _, h, w = images.shape
    for i in range(batch_size):
        for _ in range(pixel_count):
            x, y = np.random.randint(0, h), np.random.randint(0, w)
            perturbed[i, :, x, y] = torch.rand(3)
    return perturbed

In [3]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [3]:
import torch.nn as nn
from timm import create_model  # requires timm: pip install timm

class RobustHybridModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.model = create_model(
            'deit_small_patch16_224',   # Vision Transformer Small
            pretrained=True,
            num_classes=num_classes,
            img_size=128                # Adjust for 128x128 input images
        )

    def forward(self, x):
        return self.model(x)

In [4]:
# --- Adversarial Training Loop ---
def robust_train(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct = 0, 0
    for x, y in tqdm(loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        
        # Generate adversarial examples
        with torch.enable_grad():
            x_fgsm = fgsm_attack(model, x, y)
            x_pgd = pgd_attack(model, x, y)
        
        x_pixel = one_pixel_attack(x)
        
        # Combined training batch
        mixed_x = torch.cat([x, x_fgsm, x_pgd, x_pixel])
        mixed_y = torch.cat([y]*4)
        
        optimizer.zero_grad()
        outputs = model(mixed_x)
        loss = criterion(outputs, mixed_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        correct += (outputs.argmax(1) == mixed_y).sum().item()
    
    acc = correct / (4 * len(loader.dataset))
    return running_loss / len(loader), acc

In [5]:
# --- Evaluation with Attacks ---
def adversarial_test(model, loader):
    model.eval()
    results = {'clean': {'correct': 0, 'total': 0},
               'fgsm': {'correct': 0, 'total': 0},
               'pgd': {'correct': 0, 'total': 0},
               'pixel': {'correct': 0, 'total': 0}}
    
    for x, y in tqdm(loader, desc="Testing"):
        x, y = x.to(device), y.to(device)
        
        # Clean samples
        with torch.no_grad():
            out_clean = model(x)
        results['clean']['correct'] += (out_clean.argmax(1) == y).sum().item()
        results['clean']['total'] += y.size(0)
        
        # Generate attacks
        with torch.enable_grad():
            x_fgsm = fgsm_attack(model, x, y)
            x_pgd = pgd_attack(model, x, y)
        x_pixel = one_pixel_attack(x)
        
        # Test attacks
        with torch.no_grad():
            for name, data in [('fgsm', x_fgsm), ('pgd', x_pgd), ('pixel', x_pixel)]:
                out = model(data)
                results[name]['correct'] += (out.argmax(1) == y).sum().item()
                results[name]['total'] += y.size(0)
    
    # Calculate accuracies
    metrics = {}
    for key in results:
        metrics[key] = results[key]['correct'] / results[key]['total']
    return metrics

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cifake-real-and-ai-generated-synthetic-images


In [7]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train', transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test', transform=transform)

# Remap targets: FAKE=1, REAL=0
for dataset in [train_dataset, test_dataset]:
    dataset.targets = [1 if x == dataset.class_to_idx['FAKE'] else 0 for x in dataset.targets]

In [8]:
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)

In [12]:
# Model Setup
model = RobustHybridModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Training
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss, train_acc = robust_train(model, train_loader, optimizer, criterion)
    print(f"Train Acc: {train_acc:.4f}")
    
    # Periodic evaluation
    if (epoch+1) % 2 == 0:
        metrics = adversarial_test(model, test_loader)
        print("\nAdversarial Performance:")
        for attack, acc in metrics.items():
            print(f"{attack.upper():<6} Accuracy: {acc:.2%}")


Epoch 1/6


Training: 100%|██████████| 1563/1563 [37:22<00:00,  1.43s/it]


Train Acc: 0.6172

Epoch 2/6


Training: 100%|██████████| 1563/1563 [37:13<00:00,  1.43s/it]


Train Acc: 0.7662


...

Train Acc: 0.8616

Epoch 6/6


Training: 100%|██████████| 1563/1563 [37:29<00:00,  1.44s/it]


Train Acc: 0.8788


Testing: 100%|██████████| 313/313 [06:01<00:00,  1.15s/it]


Adversarial Performance:
CLEAN  Accuracy: 95.17%
FGSM   Accuracy: 75.63%
PGD    Accuracy: 72.51%
PIXEL  Accuracy: 95.17%


In [13]:
# Final Evaluation
print("\nFinal Test Results:")
metrics = adversarial_test(model, test_loader)
for attack, acc in metrics.items():
    print(f"{attack.upper():<6} Accuracy: {acc:.2%}") 

# Save Model
torch.save(model.state_dict(), "robust_hybrid_cifake.pth")


Final Test Results:


Testing: 100%|██████████| 313/313 [05:43<00:00,  1.10s/it]

CLEAN  Accuracy: 95.17%
FGSM   Accuracy: 75.63%
PGD    Accuracy: 72.51%
PIXEL  Accuracy: 95.17%
